**1. 준비 (GPU T4로 세팅)**

In [1]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers" "datasets==3.0.1" "torch==2.3.0+cu118" "torchvision==0.18.0+cu118" "torchaudio==2.3.0+cu118" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate" -f https://download.pytorch.org/whl/torch_stable.html

GPU 0: Tesla T4 (UUID: GPU-f00c2014-8cc9-0f55-3172-821b1aeda771)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 

**2. Github Link 추가하여 데이터 불러오기, 데이터 파일 명만 추가하여 파일 구분**

In [ ]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/jinu/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/jinu/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/jinu/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/jinu/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/jinu/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

In [3]:
df_all.head()

,time_text,timestamp,author,message,amount,format,date
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604


**3. 데이터 컬럼 선택 및 클리닝**

클리닝
- 디시인사이드의 경우 "dc official App" 삭제
- 해시태그, 멘션, URL, 공백 삭제
- "ㅋㅋㅋ"나 "!!!" 등의 표현 수는 남겨두기

**추후논의하여 클리닝 수정 가능

In [4]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

In [ ]:
#결과 확인
df_all.head()

,time_text,timestamp,author,message,amount,format,date,datetime_utc,datetime_kst
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604,2025-06-04 05:20:50,2025-06-04 14:20:50+09:00
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604,2025-06-04 05:32:59,2025-06-04 14:32:59+09:00
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604,2025-06-04 05:42:30,2025-06-04 14:42:30+09:00
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604,2025-06-04 05:49:39,2025-06-04 14:49:39+09:00
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604,2025-06-04 05:49:57,2025-06-04 14:49:57+09:00


In [5]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------------------
# 데이터 클리닝

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    # 1) URL 제거
    s = re.sub(r"http\S+|www\.\S+", " ", s)

    # 2) 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)

    # 3) 반복 문자 축약 (ㅋㅋㅋㅋ -> ㅋㅋ, 아아아아 -> 아아, !!!!! -> !!)
    s = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!?.])\1{2,}", r"\1\1", s)

    #4) 이모지/특수문자 과도한 것 정리 (한글/영문/숫자/기본문장부호만 유지)
    s = re.sub(r"[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ!?.]", " ", s, flags=re.UNICODE)

    # 생략 5) 팀명/드라이버 약칭 정규화 (원하면 계속 추가)
    # team_map = {
    #    "T1": "티원",
    #    "GEN": "젠지",
    #    "VER": "페르스타펜",
    #    "HAM": "해밀턴",
    #}
    #for short_name, full_name in team_map.items():
    #    s = re.sub(fr"\b{short_name}\b", full_name, s, flags=re.IGNORECASE)

    # 6) 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

####클리닝은 논의 후 추가

# ---------------------------------------------------------------------
print("Before cleaning, rows:", len(df_all))
# 데이터프레임 컬럼 선택 (df_all에서 필요한 컬럼 선택)
df = df_all[["datetime_kst", "author", "message", "format"]].copy()

# 채팅 제한 봇 데이터 삭제 (유저id: UCSvjQBDgYDB5TGVmCZObcwA)
df = df[df['author'] != 'UCSvjQBDgYDB5TGVmCZObcwA'].copy()

# 클리닝
df["clean_text"] = df["message"].map(normalize_text)


# 빈 텍스트/공백만 남은 행 제거
df["clean_text"] = df["clean_text"].fillna("").str.strip()
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After cleaning, rows:", len(df))


Before cleaning, rows: 1520294
After cleaning, rows: 1504522


In [ ]:
#데이터 확인
df.sample(5)

,datetime_kst,author,message,format,clean_text
1461601,2025-09-10 21:22:26+09:00,UCh7Z-WIQi_zjvKJE0XSli2A,대리아,PLAYOFF,대리아
756033,2025-06-13 19:10:09+09:00,UCC14QhgMZzqZJ201FrywEEg,셰셰!!!!!,RTM,셰셰!!
415830,2025-04-25 20:46:00+09:00,UCg-NM5NOb4HJZeaoBl4MkAA,도란아 보여주자 그라가스 가자,R1~2,도란아 보여주자 그라가스 가자
84507,2025-05-25 20:16:31+09:00,UCtoXKEdR0gzAD8u8I2RSr9A,방금 한정 데상혁이네,R1~2,방금 한정 데상혁이네
322828,2025-05-04 18:59:29+09:00,UCidWwf4ou61EIYgHVtn7JAA,두두 두드 ㄷㄷ,R1~2,두두 두드 ㄷㄷ


**4. 파인튜닝을 위한 데이터셋 불러오기**

사용데이터: KOTE
- KOTE: Korean Online That-gul Emotions Dataset https://github.com/searle-j/KOTE

In [6]:
from datasets import load_dataset

dataset = load_dataset("searle-j/kote", trust_remote_code=True)
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


kote.py: 0.00B [00:00, ?B/s]

README.md:   0%|          | 0.00/326 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
})


In [ ]:
# 6개의 새로운 감정 레이블 정의
new_labels = ['행복', '슬픔', '분노', '공포', '놀람', '혐오']
new_id2label = {i: label for i, label in enumerate(new_labels)}
new_label2id = {label: i for i, label in enumerate(new_labels)}

# 기존 44개 라벨을 6개로 매핑하는 딕셔너리
old_labels_list = ['불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함',
          '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움',
          '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남',
          '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음',
          '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰']
old_label2id = {label: i for i, label in enumerate(old_labels_list)}

# old_label_index -> new_label_name
label_map = {
    # 행복
    old_label2id['환영/호의']: '행복', old_label2id['감동/감탄']: '행복', old_label2id['고마움']: '행복',
    old_label2id['존경']: '행복', old_label2id['기대감']: '행복', old_label2id['뿌듯함']: '행복',
    old_label2id['편안/쾌적']: '행복', old_label2id['아껴주는']: '행복', old_label2id['즐거움/신남']: '행복',
    old_label2id['깨달음']: '행복', old_label2id['흐뭇함(귀여움/예쁨)']: '행복', old_label2id['행복']: '행복',
    old_label2id['기쁨']: '행복', old_label2id['안심/신뢰']: '행복',
    # 슬픔
    old_label2id['슬픔']: '슬픔', old_label2id['안타까움/실망']: '슬픔', old_label2id['절망']: '슬픔',
    old_label2id['서러움']: '슬픔', old_label2id['불쌍함/연민']: '슬픔',
    # 분노
    old_label2id['불평/불만']: '분노', old_label2id['화남/분노']: '분노', old_label2id['짜증']: '분노',
    old_label2id['증오/혐오']: '분노',
    # 공포
    old_label2id['공포/무서움']: '공포', old_label2id['불안/걱정']: '공포',
    # 놀람
    old_label2id['신기함/관심']: '놀람', old_label2id['당황/난처']: '놀람', old_label2id['경악']: '놀람',
    old_label2id['놀람']: '놀람',
    # 혐오
    old_label2id['지긋지긋']: '혐오', old_label2id['우쭐댐/무시함']: '혐오', old_label2id['비장함']: '혐오',
    old_label2id['의심/불신']: '혐오', old_label2id['부끄러움']: '혐오', old_label2id['한심함']: '혐오',
    old_label2id['역겨움/징그러움']: '혐오', old_label2id['어이없음']: '혐오', old_label2id['패배/자기혐오']: '혐오',
    old_label2id['귀찮음']: '혐오', old_label2id['힘듦/지침']: '혐오', old_label2id['죄책감']: '혐오',
    old_label2id['부담/안_내킴']: '혐오', old_label2id['재미없음']: '혐오',
}
ignore_label_idx = old_label2id['없음']

print("New label mapping and new labels are defined.")

**5. KcELECTRA 모델 불러오기**

**6. KcELECTRA Model Tokenizing & Training(Finetuning)**

In [ ]:
from datasets import load_dataset, Features, Value, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

# Clear GPU cache
torch.cuda.empty_cache()

# Load model and tokenizer
model_checkpoint = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load model for single-label classification with 6 labels
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(new_labels),
    id2label=new_id2label,
    label2id=new_label2id,
    use_safetensors=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Preprocess dataset: remap labels to 6 classes and filter
def remap_and_filter_dataset(example):
    new_label = -1 # Default to -1 for invalid
    for old_label_idx in example['labels']:
        if old_label_idx != ignore_label_idx and old_label_idx in label_map:
            new_label_name = label_map[old_label_idx]
            new_label = new_label2id[new_label_name]
            break # Take the first valid label found
    example['labels'] = new_label
    return example

dataset_remapped = dataset.map(remap_and_filter_dataset)
dataset_filtered = dataset_remapped.filter(lambda example: example['labels'] != -1)

print(f"Original dataset size: {len(dataset['train'])}")
print(f"Filtered dataset size: {len(dataset_filtered['train'])}")

# Tokenize text
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

tokenized_dataset = dataset_filtered.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels']
)

# Splits
train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['validation']

# Metrics for single-label classification
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    
    return {'accuracy': acc, 'f1_weighted': f1}

# Training config
training_args = TrainingArguments(
    output_dir='./kote_single_label_results',
    num_train_epochs=3,
    per_device_train_batch_size=8, # Adjusted for Colab T4
    gradient_accumulation_steps=4, # Effective batch size = 32
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1_weighted',
    greater_is_better=True,
    fp16=True,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Starting single-label model training...")
trainer.train()
trainer.save_model('./kote_single_label_trained_model')
print("✓ Single-label training completed!")


**7. Sentiment Analysis (Single Labeling)**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax
import numpy as np
from tqdm import tqdm

# Load tokenizer from ORIGINAL model (it's unchanged)
tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base-v2022')
print("✓ Tokenizer loaded from original KcELECTRA")

# Load YOUR fine-tuned single-label model weights
model_path = './kote_single_label_trained_model'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
print("✓ Your fine-tuned single-label model loaded")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Now you can use it for predictions
def predict_emotion_batch(texts, batch_size=32):
    all_predictions = []
    all_probabilities = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Predicting"):
        batch_texts = texts[i:i + batch_size]

        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            max_length=256,
            padding=True
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits
            probs = softmax(logits, dim=-1)
            predictions = torch.argmax(probs, dim=-1)

        all_predictions.extend(predictions.cpu().numpy())
        all_probabilities.extend(probs.cpu().numpy())

    return all_predictions, all_probabilities

# Apply to your dataframe
print("Starting predictions on your dataframe...")
predictions, probabilities = predict_emotion_batch(
    df['clean_text'].tolist(),
    batch_size=64 # Can use larger batch for inference
)

df['emotion_id'] = predictions
df['emotion'] = df['emotion_id'].apply(lambda x: new_id2label[x])
df['emotion_probabilities'] = [prob.tolist() for prob in probabilities]

print("✓ Predictions completed!")
print(f"\nSample results:")
print(df[['clean_text', 'emotion']].head())


In [ ]:
# Save the dataframe with single label predictions to CSV
output_path = '/content/drive/MyDrive/emotion_single_label_predictions.csv'

# Save the required columns
df_to_save = df[['datetime_kst', 'author', 'clean_text', 'emotion', 'emotion_id', 'emotion_probabilities']]
df_to_save.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✓ Results saved to {output_path}")